In [ ]:
import streamlit as st
import psutil
import joblib
import pandas as pd
import plotly.express as px
import time
from datetime import datetime
import os
model = joblib.load("laptop_diagnosis_model.pkl")
le = joblib.load("issue_label_encoder.pkl")

issue_details = {
    "High CPU Usage": "Your CPU is heavily used. Try closing tabs, apps, or scanning for malware.",
    "High RAM Usage": "RAM usage is high. Restart apps or consider a RAM upgrade.",
    "Battery Issue": "Battery is low or unhealthy. Calibrate or consider replacement.",
    "Disk Almost Full": "Disk space is nearly full. Clear junk files or upgrade your disk.",
    "Healthy": "Your system is working perfectly!"
}

st.set_page_config(page_title="Laptop Health Monitor", layout="wide")
st.title("TrackTop")
st.subheader("Real-time insights in 30-second intervals")

track_duration = st.slider("Select duration (seconds):", 10, 60, 30)
data_log = []

progress = st.progress(0)
status = st.empty()

for i in range(track_duration):
    cpu = psutil.cpu_percent(interval=1)
    ram = psutil.virtual_memory().percent
    disk = psutil.disk_usage('/').percent
    battery_info = psutil.sensors_battery()
    battery = battery_info.percent if battery_info else 50
    now = datetime.now().strftime("%H:%M:%S")

    data_log.append({
        "Time": now,
        "CPU": cpu,
        "RAM": ram,
        "Disk": disk,
        "Battery": battery
    })

    status.text(f"Tracking usage... {i+1} sec")
    progress.progress((i+1)/track_duration)

df = pd.DataFrame(data_log)

col1, col2 = st.columns(2)
with col1:
    st.line_chart(df.set_index("Time")[["CPU", "RAM"]])
with col2:
    st.line_chart(df.set_index("Time")[["Disk", "Battery"]])

st.divider()
st.subheader("Diagnose Now")

latest_values = df.iloc[-1][["CPU", "RAM", "Disk", "Battery"]].values.tolist()
input_df = pd.DataFrame([latest_values], columns=["cpu_usage_percent", "ram_used_percent", "disk_used_percent", "battery_percent"])

pred_encoded = model.predict(input_df)[0]
pred_label = le.inverse_transform([pred_encoded])[0]
explanation = issue_details.get(pred_label, "No explanation available.")

if pred_label != "Healthy":
    st.error(f"Warning: {pred_label}")
else:
    st.success("Your system appears healthy!")

st.info(f"Recommendation: {explanation}")
if not os.path.exists("scan_history.csv"):
    history_df = pd.DataFrame(columns=["Timestamp", "CPU", "RAM", "Disk", "Battery", "Issue"])
else:
    history_df = pd.read_csv("scan_history.csv")

new_entry = pd.DataFrame([{
    "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "CPU": latest_values[0],
    "RAM": latest_values[1],
    "Disk": latest_values[2],
    "Battery": latest_values[3],
    "Issue": pred_label
}])

history_df = pd.concat([history_df, new_entry], ignore_index=True)
history_df.to_csv("scan_history.csv", index=False)

st.success("Scan saved to history!")

if st.checkbox("Show Past Scan History"):
    st.dataframe(history_df.tail(10))